# 1. Inicjalizacja API
W pierwszej komórce wskazujemy adres serwera cloudberry, a w drugiej tworzymy metadane opisujące eksperyment dla którego dane chcemy zaimportować.

In [1]:
# Define Cloudberry configuration

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

In [2]:
# Create metadata necessary for CSV upload - Experiment & Configuration

meta_api = cb.Metadata(cb_config)
experiment_api = meta_api.experiment_api()
configuration_api = meta_api.experiment_configuration_api()

csv_experiment_name = 'CSV Scenario Experiment'
csv_configuration_name = 'CSV Scenario Configuration'

# optionally pass parameters to save experiment with
csv_experiment = experiment_api.find_or_create(csv_experiment_name, parameters = {})
csv_configuration = configuration_api.find_or_create(csv_experiment, csv_configuration_name, parameters={'version': 1})

Właściwy import danych z przykładowego pliku CSV:

In [3]:
# CSV-like Data Upload API

csv_api = cb.CsvFileUploader(cb_config)

csv_file_path = './data/csv_evolution_data.csv'
csv_details = cb.CsvUploadDetails(
    tags_names=['ISLAND_ID'],
    raw_fields=['BEST_SOLUTION'],
    configuration=csv_configuration,
    computation=None, # None for new one, otherwise append data to existing computation .
)

csv_computation = csv_api.upload_file(
    csv_file_path,
    csv_experiment_name,
    csv_details
)

Weryfikacja zaimportowanych danych:

In [4]:
data_api = cb.Data(cb_config)
filters = cb.DataFilters(tags={}, computation=csv_computation)
series = data_api.get_data(filters)
series.as_data_frame

,_time,ISLAND_ID,_measurement,computationId,AVERAGE_MERIT_FACTOR_VALUE,BEST_SOLUTION,WHOLE_ENERGY,STEP,AGENTS_NUMBER,BEST_MERIT_FACTOR_VALUE,series_name
0,0.000,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.178338,0000100101101101110111001110000111101000110111...,100000.000,0.0,100.0,2.598752,raw_data
1,1.524,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.134079,0000100101101101110111001110000111101000110111...,100000.000,5.0,736.0,2.598752,raw_data
2,2.796,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.130674,0000100101101101110111001110000111101000110111...,99999.610,6.0,1104.0,2.598752,raw_data
3,5.357,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.128743,0000100101101101110111001110000111101000110111...,99999.740,7.0,1656.0,2.598752,raw_data
4,10.581,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.127798,0000100101101101110111001110000111101000110111...,99999.766,8.0,2484.0,2.598752,raw_data
5,23.211,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.127996,0001101111000011110110011100010011011111101110...,99999.980,9.0,3716.0,2.783964,raw_data
6,41.554,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.125509,0001101111000011110110011100010011011111101110...,100000.340,10.0,5389.0,2.783964,raw_data
7,68.192,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.128620,0001101111000011110110011100010011011111101110...,99999.680,11.0,7044.0,2.783964,raw_data
8,110.047,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.132329,0001101111000011110110011100010011011111101110...,100000.810,12.0,8185.0,2.783964,raw_data
9,170.363,1,cloudberry-default,5fc3f896dac394387f8ba97e,1.149185,0001101111000011110110011100010011011111101110...,99999.150,13.0,8914.0,2.783964,raw_data


Następnie możemy stworzyć wykres dla wybranego parametru.
Opcjonalnie odfiltrowujemy wyniki - w przykładzie poniżej interesuje nas tylko wyspa o ID 1.

In [5]:
cb_computations = cb.ComputationSeries(cb_config)

field = 'WHOLE_ENERGY'
data_filters = cb.DataFilters(
    tags={
        'ISLAND_ID': 1
    }
)

series_pack = cb_computations.for_computations(
    computations=[csv_computation],
    field_name=field,
    api_params=None,
    data_filters=data_filters
)

Poniżej konwertujemy wynik zapytania API do modelu domenowego części biblioteki odpowiedzialnej za tworzenie wykresów.

In [6]:
from cloudberry.converters import SeriesConverter

plot_pack = SeriesConverter.data_series_pack_to_plot_series_pack(
    ds=series_pack,
    x_field="_time",
    y_field=field,
)

Konfigurujemy wykres i budujemy go przekazując do PlotBuilder'a wynik konwersji

In [12]:
import cloudberry.plots as cp

props = cp.PlotProperties.default()
props.title = "Whole energy over time"
props.show_title = True
props.show_averages = False # hide because it is 1:1 base serie
props.x_axis_name = "TIME"
props.y_axis_name = field
props.default_series_kind = cp.PlotSeriesKind.LINE

In [ ]:
builder = cp.PlotBuilder(props)
builder.add_pack(plot_pack)
plot = builder.plot()
plot